### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are the key pros of using AutoGen for your AI Agent project:

1. **Scalability**: AutoGen allows for the creation of scalable systems, enabling agents to handle multiple complex tasks simultaneously.

2. **Customization**: The framework supports high levels of customization, allowing you to build tailored AI agents that can manage specific use cases effectively.

3. **Complexity Handling**: AutoGen excels in managing complex multi-agent interactions, allowing different agents to collaborate effectively on intricate tasks that may be beyond the capability of a single AI model.

4. **Ease of Use**: Integrated tools for observability and debugging simplify monitoring and controlling agent workflows, making it easier to manage the entire system.

5. **Compatibility**: AutoGen is designed to work seamlessly with various LLMs (Language Learning Models) and toolchains, enhancing flexibility in task execution.

6. **Dynamic Automation**: It is particularly beneficial for complex automation and dynamic environments, improving efficiency and adaptability.

7. **Reduced Human Oversight**: AutoGen enables the proactive management of workflows and optimization processes, often reducing the need for constant human intervention.

These advantages make AutoGen a strong contender for enabling sophisticated AI agent functionalities in your project.

TERMINATE

## Cons of AutoGen:
Here are some cons of using AutoGen for your AI Agent project:

1. **Steep Learning Curve**: AutoGen has a more challenging learning curve due to its open-source nature and the complexity of its setup, which may be overwhelming for less experienced developers.

2. **Documentation Issues**: The documentation for AutoGen is often cited as difficult to understand, lacking clear examples and detailed guidance, which can hinder effective implementation.

3. **Limited Features**: Compared to some competitors, AutoGen may lack certain advanced features, such as a visual builder or no-code interfaces, which could make it less accessible for non-developers.

4. **Potential Technical Issues**: There are reported instances of certain features, like structured outputs, not functioning as expected, which can lead to frustration and increased development time.

5. **Complexity in Prompting**: The need for detailed and algorithmic prompts can be time-consuming and may require significant effort to create, potentially adding to project costs.

6. **Lack of Differentiation**: AutoGen may struggle to stand out from other similar tools like AG2, which could impact its appeal and support community.

7. **Resource Monitoring**: If using AutoGen in a scalable environment, there might be concerns over resource management and monitoring credits to avoid inefficiencies.

These factors should be carefully considered when deciding whether to use AutoGen in your project. 

TERMINATE



## Decision:

Based on the research provided, I recommend using AutoGen for the project. The key strengths, such as scalability, customization, and effective handling of complex multi-agent interactions, present significant advantages that align well with the project's goals. While there are notable cons, including a steep learning curve and documentation issues, the potential for dynamic automation and reduced human oversight are compelling benefits that could greatly improve project outcomes. The pros appear to outweigh the cons, especially for a project that requires sophisticated AI agent functionalities.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()